In [ ]:
import itertools
import math
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

pd.set_option('display.max_column', None)

# Context

In [ ]:
data = pd.read_csv("../data/train.csv")
data.head()
# Notice: embarked 	Port of Embarkation 	C = Cherbourg, Q = Queenstown, S = Southampton
# Notice: sibsp 	# of siblings / spouses aboard the Titanic
# Notice: parch 	# of parents / children aboard the Titanic

# Data quality assessment

In [ ]:
data.info()

## Перечисления

In [ ]:
assert data['PassengerId'].is_unique
assert 0 == data[~data['Survived'].isin((0, 1))].size
data['Survived'] = data['Survived'].astype('bool')
assert 0 == data[~data['Sex'].isin(('male', 'female'))].size
assert 0 == data[~data['Embarked'].isin(['C', 'Q', 'S', np.nan])].size
assert 0 == data[~data['Pclass'].isin([1, 2, 3])].size

## Проверим возможные полные дубликаты

In [ ]:
assert 0 == data[data.duplicated(subset=list(filter(lambda c: c != 'PassengerId', data.columns)))].size

## Родственные связи
У супружеской пары без детей или у пары сиблингов в 'SibSp' должно быть по единице, следовательно, при согласованных данных количество семей из N человек можно вычислить как M / (SibSp + 1), где M - количество записей SibSp == N - 1.

In [ ]:
_d = data.groupby('SibSp').count()['PassengerId'].reset_index()
_d[_d['PassengerId'] % (_d['SibSp'] + 1) != 0]

Вывод: для кого-то количество супругов/сиблингов указанно с ошибкой.
Аналогично проверим родиетелй/детей.

In [ ]:
_d = data.groupby('Parch').count()['PassengerId'].reset_index()
_d[_d['PassengerId'] % (_d['Parch'] + 1) != 0]

Тоже не сходится.

## Разберём сложные строки: буквенные коды могут быть полезны

In [ ]:
data[['TicketPref', 'TicketNum']] = data['Ticket'].str.extract(r'(?:(.+)\s)?(\d+)')
# Нулевых билетов нет, поэтому можно заменить NaN на 0
# TODO: data[data['TicketNum'].isnull()] = 0
data['TicketNum'] = data['TicketNum'].astype('float64')
data[['CabinPref', 'CabinNum']] = data['Cabin'].str.extract(r'([A-Za-z])(\d+)')
# Нулевых кают нет, поэтому можно заменить NaN на 0
# TODO: data[data['CabinNum'].isnull()] = 0
data['CabinNum'] = data['CabinNum'].astype('float64')

Поскольку здесь нет полностью бесполезных строк, ничего удалять не будем. При необходимости данные будем фильтровывать, исключая кортежи с пустыми значениями.
Для работы с возрастом может быть интересна классификация на взрослый/ребёнок.

In [ ]:
def get_type(age: int, sex: str) -> str:
    if age >= 60:
        a = 'Old'
    elif age >= 18:
        a = 'Adult'
    else:
        a = 'Child'
    s = sex.capitalize()
    return f'{a}{s}'

data['type'] = data.apply(lambda x: get_type(x.Age, x.Sex), axis=1)

# Data exploration

## Отдельные параметры

In [ ]:
data.describe()

Явно аномальных значений (например, возраст в 1000 лет) нет.

In [ ]:
corr_b_fields = ['Survived']
corr_n_fields = ['Pclass', 'Age', 'Parch', 'SibSp', 'Fare', 'TicketNum', 'CabinNum']
corr_o_fields = ['TicketPref', 'CabinPref', 'Embarked', 'type']

for f in corr_o_fields:
    plt.subplots()
    sns.countplot(data=data[data[f].notnull()], x=f)
for f in corr_n_fields:
    plt.subplots()
    sns.histplot(data=data[data[f].notnull()], x=f)

Здесь речь о тех пассажирах для которых известен параметр.
Префикс каюты - вероятно палуба. Числовая часть билетов явно имеет повторы, причины неизвестны. Стоимость проезда имеет три выраженных пика, вероятно, связана с пассажирским классом. Интересны пики в гистограмме возраста, стоит рассмотреть связи. Наибольшее количество пассажиров в диапазоне 20-30 лет. Больше всего пассажиров зашло на борт в Саутгемптоне. Преобладающий тип пассажира - взрослый мужчина, класс - третий.

## Взаимосвязь параметров
Рассмотрим корреляцию разных переменных. Участки кода, отображающие все возможные сочетания закомментированы, чтобы не перегружать отчёт менее малоинформативными графиками.

In [ ]:
data.corr(method="pearson")

Наблюдаем слабую связь выживаемости с возрастом (0.103895), стоимостью билета (0.187534) и среднюю связь с пассажирским классом (-0.289723).
Стоимость билета сильно связана (-0.417354) с пассажирским классом (очевидно, 1-й дороже).

In [ ]:
for a, b in itertools.combinations(('Pclass', 'CabinPref', 'Embarked', 'Survived', 'Sex'), 2):
    sns.catplot(y=a, hue=b, kind="count", data=data)
for o in ('Pclass', 'CabinPref', 'Embarked', 'Survived', 'Sex'):
    for n in ('Age', 'Parch', 'SibSp', 'Fare'):
        plt.subplots()
        sns.boxplot(data=data, y=n, x=o)

Большинство пассажиров из Саутгемптона.
Больше всего выжило в первом классе, меньше - третьем.
Аналогично по классам распределены женщины.
Самый маленький процент выживших на палубе A - вдвое меньше чем погибших, на остальных палубах выживших больше чем погибших. Это совпадает с распределением женщин по палубам.
Аналогичные соотношения и в пункте отправления.
В первом классе едут люди постарше, семейные (только в нём), третий класс выбирают пассажиры, едущие в одиночку.
Стоимость проезда в первом классе выше.
Почему-то для мужчин не указывается количество родственников.

### Стоимость проезда

In [ ]:
plt.subplots(figsize=(10,10))
sns.boxplot(data=data, y='Fare', x='CabinPref', hue='Pclass')
plt.subplots(figsize=(10,10))
sns.boxplot(data=data, y='Fare', x='type', hue='Pclass')
plt.subplots(figsize=(10,10))
sns.boxplot(data=data, y='Fare', x='Embarked', hue='Pclass')

Даже с учётом обилия выбросов хорошо видно, что стоимость проезда и её вариативность возрастают вместе с комфортабельностью класса - предположительно возрастает количество доступных для приобретения дополнительных услуг, причём для женщин всех возрастов приобретается больший набор.
Доступный набор услуг не отличается от пункта отправления (хорошо видно по стоимости проезда второго и третьего классов) и можно предположить, что из Шербурга плывут более состоятельные пассажиры обоих полов.

### Выживаемость
Стоит рассмотреть соблюдение принципа спасения "женщины и дети в первую очередь".

In [ ]:
sns.catplot(y='type', hue='Survived', kind="count", data=data)

In [ ]:
plt.subplots(figsize=(14, 4))
sns.heatmap(data.groupby(['CabinPref', 'Pclass', 'Sex']).count()
            .reset_index().pivot(columns=['CabinPref', 'Pclass'], index='Sex', values='PassengerId'), annot=True, fmt='.0f')
plt.subplots(figsize=(14, 4))
sns.heatmap((data[data['Survived']].groupby(['CabinPref', 'Pclass', 'Sex']).count()
             / data.groupby(['CabinPref', 'Pclass', 'Sex']).count())
            .reset_index().pivot(columns=['CabinPref', 'Pclass'], index='Sex', values='PassengerId'), annot=True, fmt='.0%')

Во всех возрастных категориях (и среди детей) предпочтение по спасению в пользу женщин.
На палубе T слишком мало пассажиров, чтобы судить об удобстве эвакуции с неё. Хуэе всего для эвакуации приспособлена палуба A.

### Бонус. Спасают ли от утопления "счастливые билеты"?

In [ ]:
lucky_data = data[data['TicketNum'].notnull()][['Survived', 'TicketNum']]
def is_lucky(n):
    n = str(int(n))
    return sum(map(int, n[:math.floor(len(n)/2)])) == sum(map(int, n[math.ceil(len(n)/2):]))

lucky_data['is_lucky'] = lucky_data['TicketNum'].apply(is_lucky)
lucky_data = lucky_data.groupby(['is_lucky', 'Survived']).count().reset_index()
lucky_data[lucky_data['Survived']].reset_index()['TicketNum'] / lucky_data.groupby('is_lucky').sum().reset_index()['TicketNum']

Среди обладателей счастливых и обычных билетов процент выживших одинаков - "счастливые билетики" от утопления *не* спасают.

# Summary
Наибольшие шансы на спасение (>=90%) у женщин, едущих вторым классом на палубах D, E, F, G. Наименьшие шансы на спасение у мужчин, едущих третьим классом на палубе A. Самая "опасная" палуба - A.